In [1]:
from label_pipeline2 import InteriorImageClassifier, generate_class_label2ref_images
from config import MODEL_NAME, BASE_PROMPT_EN, BASE_PROMPT_RU
import os
from pathlib import Path
from PIL import Image
import csv
from tqdm import tqdm
import shutil

In [2]:
def process_directory(
        classifier: InteriorImageClassifier,
        image_dir: Path,
        output_dir: Path,
        ref_images_dir: Path | None = None,
        output_csv: Path = Path("results.csv"),
        extensions: tuple = (".jpg", ".png", ".jpeg")
    ) -> None:
    """
    Обрабатывает изображения и сортирует по папкам классов
    с постепенной записью результатов в CSV.
    """
    # Создаем корневую директорию для результатов
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Путь к итоговому CSV
    csv_path = output_dir / output_csv
    
    # Заголовки CSV
    fieldnames = ["image_filename", "class", "raw_response"]
    
    # Открываем CSV для постепенной записи
    with csv_path.open(mode='w', newline='', encoding='utf-8') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        
        # Обрабатываем каждое изображение
        img_files = [f for f in image_dir.iterdir() if f.suffix.lower() in extensions]
        for img_file in tqdm(img_files, desc="Обработка изображений"):
            class_label2ref_images = None
            if ref_images_dir is not None:
                class_label2ref_images = generate_class_label2ref_images(
                    ref_images_dirpath=ref_images_dir,
                    max_files_per_class=1
                )
            image = Image.open(img_file)
            result = classifier(image, class_label2ref_images=class_label2ref_images)
            
            # Создаем папку класса (A0, A1... ERROR)
            class_dir = output_dir / result["class"]
            class_dir.mkdir(exist_ok=True)
            
            # Копируем изображение в папку класса
            shutil.copy2(img_file, class_dir / img_file.name)
            
            # Записываем результат в CSV
            writer.writerow(result)
    
    print(f"\nГотово! Результаты в {output_dir}")

In [3]:
# Инициализация классификатора
classifier = InteriorImageClassifier(model_name=MODEL_NAME, base_prompt=BASE_PROMPT_EN)
print(classifier.pipe.torch_dtype)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.
Device set to use cuda:0


torch.bfloat16


In [4]:
current_dirpath = Path.cwd()
print(f"Директория: {current_dirpath}")
data_dirpath = current_dirpath / "data"

image_dir = data_dirpath / "cosmetic/cosmetic+"
output_dir = data_dirpath / "labeled_with_few_shot"
ref_images_dir = data_dirpath / "reference_images_resized"
ouput_csv = output_dir / "results.csv"

Директория: /home/little-garden/CodeProjects/InteriorClass


In [ ]:
# class_label2ref_images = generate_class_label2ref_images(
#     ref_images_dirpath=ref_images_dir
# )
# target_image = Image.open(data_dirpath / "cosmetic/cosmetic+/269577546_1.jpg")
# prompt = classifier.build_few_shot_prompt(
#     target_image=target_image,
#     class_label2ref_images=class_label2ref_images
# )

# content = prompt[0]['content']
# for d in content:
#     print(d)

# image = Image.open(str(data_dirpath / "cosmetic/cosmetic+/269577546_1.jpg"))
# response = classifier.classify_image(
#     image=image,
#     class_label2ref_images=class_label2ref_images
# )
# print(f"1 {response}")

# image = Image.open(str(data_dirpath / "cosmetic/cosmetic+/269577546_1.jpg"))
# response = classifier(image=image, class_label2ref_images=class_label2ref_images)
# print(f"2 {response}")

process_directory(
    classifier=classifier,
    image_dir=image_dir,
    output_dir=output_dir,
    ref_images_dir=ref_images_dir,
    output_csv=ouput_csv
)

Обработка изображений:  12%|█▏        | 1544/13426 [34:49<4:13:23,  1.28s/it]